In [ ]:
from huggingface_hub import login

# Saving Models
Be careful not to overwrite!


In [ ]:
model_repo_name = "Benw8888/lp_saes"
model_id = "Benw8888/lp_saes"
mode = "bias"
chkpt = 0
local_file = f"/root/sparsify/notebooks/trained_models/sae_l0.1_0.1.pt"
file_in_repo = (
    f"sft_sae_70m.pt"  # "sft_sae_6b.pt" "transfer_sft_ppo_6b.pt" f"transfer_sft_base_6b_{mode}.pt"
)
# each checkpoint was 8mil tokens, so chkpt 6 was 56mil tokens (0 indexed)
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj=local_file,
    path_in_repo=file_in_repo,
    repo_id=model_id,
    repo_type="model",
)

In [ ]:
from huggingface_hub import HfApi

l1_alphas = [
    1e-6,
    3e-6,
    1e-5,
    3e-5,
    1e-4,
    3e-4,
    1e-3,
    3e-3,
    1e-2,
    3e-2,
    1e-1,
    3e-1,
    1,
    3,
]
ps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1]

model_repo_name = "Benw8888/lp_saes"
model_id = "Benw8888/lp_saes"

seed = "3"
folder = f"toy_saes{seed}"
api = HfApi()
api.upload_folder(
    folder_path=f"/root/sparsify/trained_models/toy_saes{seed}",  # {folder}",
    path_in_repo=folder,
    repo_id=model_id,
    repo_type="model",
)


# Loading Models
Downloading Files


Downloading From Folders


In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import hf_hub_download
import torch
import os
import torch.nn as nn

class AutoEncoder(nn.Module):
    def __init__(self, activation_size, n_dict_components):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(nn.Linear(activation_size, n_dict_components), nn.ReLU())
        self.decoder = nn.Linear(n_dict_components, activation_size, bias=False)

        # Initialize the decoder weights orthogonally
        nn.init.orthogonal_(self.decoder.weight)

    def forward(self, x):
        c = self.encoder(x)

        # Apply unit norm constraint to the decoder weights
        self.decoder.weight.data = nn.functional.normalize(self.decoder.weight.data, dim=0)

        x_hat = self.decoder(c)
        return x_hat, c

    def get_dictionary(self):
        self.decoder.weight.data = nn.functional.normalize(self.decoder.weight.data, dim=0)
        return self.decoder.weight

    @property
    def device(self):
        return next(self.parameters()).device
    

l1_alphas = [
    1e-6,
    3e-6,
    1e-5,
    3e-5,
    1e-4,
    3e-4,
    1e-3,
    3e-3,
    1e-2,
    3e-2,
    1e-1,
    3e-1,
    1,
    3,
]
ps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1]

huggingface_folder_names = []
huggingface_folder_names += [
    "toy_saes",
    "toy_saes0",
    "toy_saes1",
]  # , "transfer_base_sft_70m"] "base_autoTED_70m"

model_repo_name = "Benw8888/lp_saes"
model_id = "Benw8888/lp_saes"

for folder in huggingface_folder_names:
    if not os.path.exists("trained_models"):
        os.makedirs("trained_models")
    if not os.path.exists(f"trained_models/{folder}"):
        os.makedirs(f"trained_models/{folder}")
    for p in ps:
        for l1 in l1_alphas:
            file_name = f"sae_l{p}_{l1}.pt"
            hf_name = f"{folder}/{file_name}"
            ae_download_location = hf_hub_download(repo_id=model_id, filename=hf_name)
            downloaded_autoencoder = torch.load(ae_download_location)
            torch.save(downloaded_autoencoder, f"trained_models/{folder}/{file_name}")
            
        file_name = f"ground_truth_features.pt"
        hf_name = f"{folder}/{file_name}"
        ae_download_location = hf_hub_download(repo_id=model_id, filename=hf_name)
        downloaded_autoencoder = torch.load(ae_download_location)
        torch.save(downloaded_autoencoder, f"trained_models/{folder}/{file_name}")
